In [1]:
import numpy as np
!pip install pysindy
from scipy.integrate import solve_ivp
from sklearn.metrics import mean_squared_error
import pysindy as ps
import statistics
from scipy.interpolate import interp1d

dt = 2
t_train = np.linspace(0, 58, 30)
# Dataset
t_train_span = (t_train[0], t_train[-1])
X = np.array([[32, 50, 12, 10, 13, 36, 15, 12, 6, 6, 65, 70, 40, 9, 20, 34, 45, 40, 15, 15, 60, 80, 26, 18, 37, 50, 35, 12, 12, 25],[20,20, 52, 83, 64, 68, 83, 12, 36, 150, 110, 60, 7, 10, 70, 100, 92, 70, 10, 11, 137, 137, 18, 22, 52, 83, 18, 10, 9, 65]])

# Cubic Interpolation
f1 = interp1d(t_train, [32, 50, 12, 10, 13, 36, 15, 12, 6, 6, 65, 70, 40, 9, 20, 34, 45, 40, 15, 15, 60, 80, 26, 18, 37, 50, 35, 12, 12, 25], kind='cubic')
f2 = interp1d(t_train, [20,20, 52, 83, 64, 68, 83, 12, 36, 150, 110, 60, 7, 10, 70, 100, 92, 70, 10, 11, 137, 137, 18, 22, 52, 83, 18, 10, 9, 65], kind='cubic')


tnew = np.linspace(0, 58, num=150, endpoint=True)
dtNew=58/150
X1=f1(tnew)
X2=f2(tnew)
X=[X1,X2]
X=X/np.max(X)

feature_names = ['x', 'y']

n_candidates_to_drop=2
N=1000  #Bagging iterations

# SINDy Library Ensemble
E_library_opt = ps.STLSQ()
model = ps.SINDy(feature_names=feature_names, optimizer=E_library_opt, discrete_time=True)
model.fit([np.transpose(X)], t=dtNew, library_ensemble=True, n_models=N, quiet=True,n_candidates_to_drop=n_candidates_to_drop, multiple_trajectories=True)
E_library_Xi = np.asarray(model.coef_list)
n_targets = len(feature_names)
n_features = len(model.get_feature_names())
inclusion_probabilities = np.count_nonzero(model.coef_list, axis=0)

output = np.var(E_library_Xi, axis=0, dtype=np.float64)
a=output>=np.max(output)
B=inclusion_probabilities/N <= 0.3 + a
inclusion_probabilities[B] = 0.0

Theta= E_library_Xi[1,:,:]
Theta[inclusion_probabilities <= 100] = 0.0

Xix1=Theta[0,0]
Xix2=Theta[0,1]
Xix3=Theta[0,2]
Xix4=Theta[0,3]
Xix5=Theta[0,4]
Xix6=Theta[0,5]   

Xiy1=Theta[1,0]
Xiy2=Theta[1,1]
Xiy3=Theta[1,2]
Xiy4=Theta[1,3]
Xiy5=Theta[1,4]
Xiy6=Theta[1,5]

for i in range(1,N):
  Theta= E_library_Xi[i,:,:]
  Theta[inclusion_probabilities <= 50] = 0.0
  Xix1=Theta[0,0]+Xix1
  Xix2=Theta[0,1]+Xix2
  Xix3=Theta[0,2]+Xix3
  Xix4=Theta[0,3]+Xix4
  Xix5=Theta[0,4]+Xix5
  Xix6=Theta[0,5]+Xix6 

  Xiy1=Theta[1,0]+Xiy1
  Xiy2=Theta[1,1]+Xiy2
  Xiy3=Theta[1,2]+Xiy3
  Xiy4=Theta[1,3]+Xiy4
  Xiy5=Theta[1,4]+Xiy5
  Xiy6=Theta[1,5]+Xiy6
  
XiF=np.array([[Xix1/1000, Xix2/1000, Xix3/1000, Xix4/1000, Xix5/1000, Xix6/1000],[Xiy1/1000, Xiy2/1000, Xiy3/1000, Xiy4/1000, Xiy5/1000, Xiy6/1000]])

chopped_inds = np.any(inclusion_probabilities != 0.0, axis=0)
chopped_inds = np.ravel(np.where(~chopped_inds))

X=X[:,50:90]

# Bagging of coefficients on the truncated library
library = ps.PolynomialLibrary(degree=2, library_ensemble=True,ensemble_indices=chopped_inds)
E_opt = ps.STLSQ()
model = ps.SINDy(feature_names=feature_names, optimizer=E_opt, feature_library=library, discrete_time=False)
model.fit([np.transpose(X)], t=dtNew, ensemble=True, n_models=N, n_subset=20, quiet=True,replace=True, multiple_trajectories=True)
d_st_ensemble_coefs = np.asarray(model.coef_list)
d_st_mean = np.mean(d_st_ensemble_coefs, axis=0)
d_st_std = np.std(d_st_ensemble_coefs, axis=0)
d_st_median = np.median(d_st_ensemble_coefs, axis=0)

# Add zeros to get coefficient matrices to original full size
for i in range(len(chopped_inds)):
    d_st_mean = np.insert(d_st_mean, chopped_inds[i], 0.0, axis=-1)
    d_st_std = np.insert(d_st_std, chopped_inds[i], 0.0, axis=-1)
    d_st_median = np.insert(d_st_median, chopped_inds[i], 0.0, axis=-1)

print(d_st_mean)
print(d_st_median)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 104 kB 9.5 MB/s 
[[ 0.         -0.59802426  0.07685026  0.          1.00809828  0.        ]
 [ 0.         -0.26595214  0.48498047  0.         -1.58126994  0.        ]]
[[ 0.         -0.59256876  0.07496656  0.          1.05202684  0.        ]
 [ 0.         -0.30187802  0.48795806  0.         -1.54275237  0.        ]]
